# FLARE Query Engine

Source: https://docs.llamaindex.ai/en/stable/examples/query_engine/flare_query_engine.html

Adapted from the paper "Active Retrieval Augmented Generation"

Currently implements FLARE Instruct, which tells the LLM to generate retrieval instructions.

# Setup

# Run from an virtualenv!
```
python3 -m virtualenv myenv 
source myenv/bin/activate 
# pip install notebook
jupyter notebook
```

In [ ]:
# Only needed, if pip-installs of the next cells do not work correctly
# !pip freeze | grep llama | xargs pip uninstall -y
# !pip install -U llama-index llama-index-llms-azure-openai llama-index-embeddings-azure-openai --force-reinstall --no-cache-dir

In [1]:
# Pydantic >=2 causes compatibility issues with llama-index - that's why it's downgraded to 1.10.10
%pip install llama-index llama-index-llms-openai pydantic==1.10.10 --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
import os
import json
from getpass import getpass
import psutil

In [4]:
IN_NOTEBOOK = any(["jupyter-notebook" in i for i in psutil.Process().parent().cmdline()])
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

Secrets (JSON string):  ········


In [5]:
os.environ["OPENAI_API_KEY"] = CREDS['OpenAI']['v2']['credential'] 

## Download Data

In [6]:
!mkdir -p 'data/paul_graham/'
!curl 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' --output 'data/paul_graham/paul_graham_essay.txt'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 75042  100 75042    0     0   224k      0 --:--:-- --:--:-- --:--:--  229k


# 1. Basic RAG

In [8]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

# load data
documents = SimpleDirectoryReader(input_dir="./data/paul_graham").load_data()

# build VectorStoreIndex that takes care of chunking documents and encoding chunks to embeddings for future retrieval
index = VectorStoreIndex.from_documents(documents=documents)

# The QueryEngine class is equipped with the generator and facilitates the retrieval and generation steps
query_engine = index.as_query_engine()

In [9]:
# Use your Default RAG
response = query_engine.query("Can you tell me about the author's trajectory in the startup world?")
print(response)

In [22]:
response = query_engine.query("Give me information about the experience of building shops for users in the provided documents.")
print(response)

Building shops for users in the provided documents was initially seen as a somewhat humiliating task, as the software was designed for users to create their own stores. However, this approach of building stores for users inadvertently provided valuable insights into the retail industry and the user experience of the software. Despite feeling initially mystified and repelled by the business aspect, the experience of directly assisting users in setting up their stores led to a deeper understanding of retail needs and user preferences. This hands-on approach not only helped in acquiring users but also shaped the direction of the software development based on real-world user interactions and feedback.


In [21]:
response = query_engine.query("List all informations about Lisp which you can find in the provided documents.")
print(response)

Lisp was interesting for its own sake and not just for its association with AI. The author decided to focus on Lisp and wrote a book about Lisp hacking called "On Lisp." The book wasn't published until 1993, but much of it was written during the author's time in grad school. The author was genuinely in love with Lisp hacking and worked on it while also being in a PhD program in computer science. Lisp is mentioned in the context as a language the author found interesting and engaging beyond its AI association.


# 3. FLARE RAG

In [5]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-4", temperature=0)
Settings.chunk_size = 512

In [7]:
# Load data
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

documents = SimpleDirectoryReader("./data/paul_graham").load_data()
index = VectorStoreIndex.from_documents(
    documents,
)

In [8]:
index_query_engine = index.as_query_engine(similarity_top_k=2)

In [9]:
from llama_index.core.query_engine import FLAREInstructQueryEngine

flare_query_engine = FLAREInstructQueryEngine(
    query_engine=index_query_engine,
    max_iterations=7,
    verbose=True,
)

In [10]:
response = flare_query_engine.query(
    "Can you tell me about the author's trajectory in the startup world?"
)

Query: Can you tell me about the author's trajectory in the startup world?
Current response: 
Lookahead response: The author first entered the startup world in [Search(When did the author first enter the startup world?)], and since then, they have [Search(What has the author achieved in the startup world?)].
Updated lookahead response: The author first entered the startup world in a time not specified, and since then, they have 
Current response:  The author first entered the startup world in a time not specified, and since then, they have
Lookahead response: been involved in several successful ventures, including [Search(What successful startups has the author been involved in?)]
Updated lookahead response: been involved in several successful ventures, including ventures that the context does not provide specific information about.
Current response: The author first entered the startup world in a time not specified, and since then, they have been involved in several successful venture

In [11]:
print(response)

The author first entered the startup world in a time not specified, and since then, they have been involved in several ventures, including startups that are not specified in the context. Some of their most notable achievements include giving a talk to the Harvard Computer Society on how to start a startup, realizing the importance of angel investing and deciding to stop procrastinating about it, and co-founding an investment firm with Jessica, Robert, and Trevor, implementing innovative ideas they had discussed. This firm, Y Combinator, was unique in that it combined the concepts of an angel firm and a startup incubator, which was a novel idea at the time. Their journey in the startup world continued as they started working on a new version of Arc in the summer of 2006. To test this new Arc, he wrote Hacker News, which was initially meant to be a news aggregator for startup founders and their influence in the startup world grew. Their influence further expanded when Hacker News grew in

In [13]:
response = flare_query_engine.query(
    "Can you tell me about what the author did during his time at YC?"
)

Query: Can you tell me about what the author did during his time at YC?
Current response: 
Lookahead response: The author was involved in [Search(What did the author do during his time at YC?)] during his time at YC.
Updated lookahead response: The author was involved in various activities during his time at YC. He worked on problems brought by new batches of startups every six months, making their issues his own. This work was engaging due to the variety of problems and the effectiveness of good founders. He also dealt with disputes between cofounders, identified when people were lying, and confronted those who mistreated the startups. Additionally, he was involved in the selection and funding process for startups, investing $6k per founder in return for 6% of the company. He also noticed the benefits of funding startups in batches, such as reducing isolation among founders and creating a tight-knit community of alumni during his time at YC.
Current response:  The author was involved 

In [14]:
print(response)

The author was involved in various activities during his time at YC. He worked on problems brought by new batches of startups every six months, making their issues his own. This work was engaging due to the variety of problems and the effectiveness of good founders. He also dealt with disputes between cofounders, identified when people were lying, and confronted those who mistreated the startups. Additionally, he was involved in the selection and funding process for startups, investing $6k per founder in return for 6% of the company. He also noticed the benefits of funding startups in batches, such as reducing isolation among founders and creating a tight-knit community of alumni during his time at YC. Done


In [ ]:
response = flare_query_engine.query(
    "Tell me about the author's life from childhood to adulthood"
)

Query: Tell me about the author's life from childhood to adulthood
Current response: 
Lookahead response: The author grew up in a small town, where they [Search(What did the author do during their childhood?)] and later went on to attend college, majoring in [Search(What did the author major in during college?)].
Updated lookahead response: The author grew up in a small town, where they mainly worked on writing and programming outside of school. They wrote short stories and tried programming on the IBM 1401 using an early version of Fortran and later went on to attend college, majoring in 
Current response:  The author grew up in a small town, where they mainly worked on writing and programming outside of school. They wrote short stories and tried programming on the IBM 1401 using an early version of Fortran and later went on to attend college, majoring in
Lookahead response: computer science and English literature. After college, they [Search(What did the author do after college?)]
Up

In [ ]:
print(response)

The author grew up in a small town, where they mainly worked on writing and programming outside of school. They wrote short stories and tried programming on the IBM 1401 using an early version of Fortran and later went on to attend college, majoring in computer science and English literature. After college, they wrote essays on various topics, worked on spam filters, did some painting, and hosted dinners for friends. They also bought a building in Cambridge to use as an office. Later, the author applied to art schools, got accepted into RISD, and attended their foundation classes. They also received an invitation to take the entrance exam at the Accademia di Belli Arti in Florence. During their time at RISD and the Accademia di Belli Arti, the author honed their artistic skills and further developed their writing, eventually transitioning into a successful career as an author and artist. The author achieved several things in their career, including publishing essays online, writing a b

In [ ]:
response = index_query_engine.query(
    "Can you tell me about the author's trajectory in the startup world?"
)

In [ ]:
print(str(response))

The author's trajectory in the startup world began with their involvement in various projects and activities, such as writing essays on different topics, working on spam filters, and painting. They also hosted dinners for friends, which helped them learn how to cook for groups and network with people from various backgrounds.

In October 2003, the author met Jessica Livingston at a party, who later became a significant figure in their startup journey. Jessica worked in marketing at a Boston investment bank and was intrigued by the stories of startup founders she met through the author. She decided to compile a book of interviews with these founders.

In early 2005, Jessica interviewed for a marketing job at a Boston VC firm, which led the author to discuss the issues with venture capital and how it could be improved. The author also gave a talk at the Harvard Computer Society about starting a startup, which made them realize they should start angel investing.

On March 11, the author, 

In [ ]:
response = index_query_engine.query(
    "Tell me about the author's life from childhood to adulthood"
)

In [ ]:
print(str(response))

The author's life from childhood to adulthood includes a variety of experiences and interests. They wrote numerous essays on various topics, which were later compiled into a book called Hackers & Painters. They also worked on spam filters and pursued painting as a hobby. The author used to host dinners for friends every Thursday night, which taught them how to cook for groups. They bought a building in Cambridge, which was a former candy factory and later a porn studio, to use as an office.

In October 2003, the author met Jessica Livingston at a party, and they started dating a few days later. Jessica worked in marketing at a Boston investment bank and later decided to compile a book of interviews with startup founders. When she was looking for a new job, the author shared their thoughts on how venture capital should be improved.

The author also attended the Accademia, a prestigious institution, to study painting. However, they were disappointed with the lack of teaching and learning